# Statistical analysis
In this example we create a series of T1 measurement each with a different set of parameters specified in settings.json.

Then we retrive a number of datasets where a T1 experiment was performed by retriving all dataset with attribute 'measurement_type'='T1'. Then we perform an analysis where we check the correlation of the 4 measurement parameter of each dataset specified in the settings.json, and we make a new dataset with a report. 

### Create several T1 measurement datasets

In [2]:
import numpy as np
import xarray as xr
from qdrive import dataset
import json

# Create several datasets with different T1 values that correlate with param_4
n_datasets = 15

for i in range(n_datasets):
    # Generate T1 value that correlates with param_4 (with some noise)
    base_param_4 = 10 + i * 5  # param_4 ranges from 10 to 55
    t1_value = 15 + 0.8 * base_param_4 + np.random.normal(0, 2)  # Linear correlation with noise
    
    # Generate T1 measurement data
    t1_times = np.linspace(0, 100, 50)  # microseconds
    t1_signal = np.exp(-t1_times/t1_value) + 0.02 * np.random.randn(50)
    
    # Create xarray dataset with units and attributes
    xr_ds = xr.Dataset({
        'Signal': (['Time'], t1_signal, {'units': 'V'})
    }, coords={
        'Time': (['Time'], t1_times, {'units': 'us'})
    }, attrs={'T1_fitted': t1_value, 'T1_units': 'us'})
    
    # Create JSON parameters with param_4 correlating to T1
    json_params = {
        'param_1': np.random.uniform(1, 10),
        'param_2': np.random.uniform(20, 50),
        'param_3': np.random.uniform(1, 100),
        'param_4': base_param_4 + np.random.normal(0, 1),  # This correlates with T1
    }
    
    # Create QHarbor dataset
    dsq = dataset.create(f'T1_measurement', 
                        scope_name=None, 
                        description=f'T1 measurement dataset {i+1} with T1≈{t1_value:.1f}μs', 
                        tags=['T1_measurement'],
                        attributes={'measurement_type': 'T1', 'sample': f'S{i+1:02d}'})


    # Add the xarray data as HDF5 file
    dsq['measurement.hdf5'] = xr_ds
    
    # Add the parameters as JSON file
    dsq['settings.json'] = json_params

### Search and analysis all recent T1 datasets

In [ ]:
# Install dependencies:
# ! pip install seaborn
# ! pip install bokeh

In [3]:
from t1_analysis import t1_correlation_analysis
from qdrive.dataset.search import search_datasets
from datetime import date

# Search for T1 datasets and return as a list.
from datetime import datetime
search_result = search_datasets(attributes={'measurement_type': 'T1'}, start_date=datetime(2025, 9, 2))
datasets = list(search_result)
print(f"Found {len(datasets)} T1 datasets.")

# Run the analysis
report_file, results_df = t1_correlation_analysis(datasets)

Found 25 T1 datasets.
Report saved to: data/t1_correlation_report.html
Analyzed 25 samples with T1 range: 20.5 - 79.4 μs
